# Entrenamiento Por columna por FB Prophet
### Pasos:
1-. Importacion de librerias
2-. Division de Dataset (Marcado de huecos sinteticos)
3-. Calibracion de Parametros
4-. Entrenamiento de ML
5-. Generacion de Graficas de datos
6-. Pruebas de Precision

In [1]:
%pip install prophet
%pip install scikit-learn
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import itertools
import os 
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric



## FB Prophet Cross-Validation

In [11]:
import logging
logging.getLogger('fbprophet').setLevel(logging.ERROR)

from itertools import product
from prophet import Prophet

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

from tqdm import tqdm



### Hyperparameter Tuning

In [5]:
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import logging

# Silenciar logs de Prophet y CmdStanPy 
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True
logging.getLogger('prophet').setLevel(logging.WARNING)


# Directorio de tus datos originales y carpetas de salida
directory = {
    'Seco': {
        'path': '../data/data_conagua_clasificada/Seco/NR_25037.csv', 
        'output_folder': './hyperparameter_tuning_results/Seco/NR_25037/'
    },
    'Templado': {
        'path': '../data/data_conagua_clasificada/Templado/NR_25033.csv', 
        'output_folder': './hyperparameter_tuning_results/Templado/NR_25033/'
    },
    'Tropical': {
        'path': '../data/data_conagua_clasificada/Tropical/NR_25046.csv', 
        'output_folder': './hyperparameter_tuning_results/Tropical/NR_25046/'
    }
}



columnas_objetivo = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

# Definir el espacio de hiperparámetros
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0]
}

# Cutoffs deseados
target_cutoffs = pd.to_datetime(['1965-06-01', '1980-02-15', '2000-08-15', '2014-02-15'])

# Generar todas las combinaciones
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

print(f"INICIANDO TUNING DE HIPERPARAMETROS ({len(all_params)} combinaciones por variable)\n")


for region, info in directory.items():
    file_route = info['path']
    output_route = info['output_folder']
    
    os.makedirs(output_route, exist_ok=True)
    
    print(f"--> Procesando Estación: {region} ({file_route})")
    
    if not os.path.exists(file_route):
        print(f"No existe el archivo: {file_route}")
        continue
        
    # Cargar Dataset
    try:
        df = pd.read_csv(file_route)
    except Exception as e:
        print(f"    [ERROR] Leyendo CSV: {e}")
        continue

    # Iterar sobre cada columna objetivo
    for columna in columnas_objetivo:
        if columna not in df.columns:
            print(f"    [SKIP] La columna {columna} no existe en el archivo.")
            continue
            
        print(f"    Optimizar Hiperparametros para: {columna}...")
        
        # Preparar datos para Prophet
        df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
        df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
        df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
        df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)
        
        if len(df_prop) < 100:
            print("       [WARN] Muy pocos datos para entrenar. Saltando.")
            continue

        #Solo usar cutoffs que estén DENTRO del rango de fechas de esta estación
        # Prophet falla si el cutoff es anterior al inicio de los datos
        min_date = df_prop['ds'].min()
        max_date = df_prop['ds'].max()
        valid_cutoffs = [c for c in target_cutoffs if c > min_date and c < max_date]
        
        if len(valid_cutoffs) == 0:
            print("       [WARN] Ningún cutoff es válido para el rango de fechas de estos datos. Usando CV por defecto.")
            use_custom_cutoffs = False
        else:
            use_custom_cutoffs = True

        rmsees = []
        
        # Iterar combinaciones de parámetros
        for params in tqdm(all_params, desc=f"       Tuning {columna}", leave=False):
            try:
                m = Prophet(**params)
                m.fit(df_prop)
                
                # Cross Validation
                if use_custom_cutoffs:
                    df_cv = cross_validation(m, cutoffs=valid_cutoffs, horizon='365 days', parallel="processes")
                else:
                    # Fallback si las fechas fijas no sirven: usar sliding window automático
                    df_cv = cross_validation(m, initial='730 days', period='365 days', horizon='365 days', parallel="processes")
                
                # Calcular métricas
                df_p = performance_metrics(df_cv, rolling_window=1)
                rmsee = df_p['rmse'].values[0]
                rmsees.append(rmsee)
                
            except Exception as e:
                # Si falla una combinación, guardamos infinito para descartarla luego
                rmsees.append(np.inf)
        
        tuning_results = pd.DataFrame(all_params)
        tuning_results['rmse'] = rmsees
        
        # Ordenar por mejor resultado
        tuning_results = tuning_results.sort_values('rmse')
        
        # Guardar CSV con los mejores parametros
        best_csv_path = os.path.join(output_route, f"best_params_{columna}.csv")
        tuning_results.to_csv(best_csv_path, index=False)
        
        # Generar Imagen de Tabla
        n_rows = 40
        display_df = tuning_results.head(n_rows).copy()

        # Formatear decimales para la imagen
        for c in display_df.select_dtypes(include=[float]).columns:
            display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "Error")

        rows, cols = display_df.shape

        fig_w = max(8, min(24, 1.2 * cols))
        fig_h = max(2, 0.3 * rows)
        
        fig, ax = plt.subplots(figsize=(fig_w, fig_h))
        ax.axis('off')

        table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.2)
        
        plt.title(f"Hyperparameter Tuning - {region} - {columna}", y=1.02)
        
        png_file = os.path.join(output_route, f"tuning_table_{columna}.png")
        fig.tight_layout()
        fig.savefig(png_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        
        print(f"       [OK] Mejor RMSE: {tuning_results.iloc[0]['rmse']:.4f}")
        print(f"       -> Tabla guardada en: {png_file}")



--- INICIANDO TUNING DE HIPERPARÁMETROS (25 combinaciones por variable) ---

--> Procesando Estación: Tropical (../data/data_conagua_clasificada/Tropical/NR_25046.csv)
    Optimizar Hiperparámetros para: PRECIP...


       [OK] Mejor RMSE: 10.0134
       -> Tabla guardada en: ./hyperparameter_tuning_results/Tropical/NR_25046/tuning_table_PRECIP.png
    Optimizar Hiperparámetros para: EVAP...


       [OK] Mejor RMSE: 1.4574
       -> Tabla guardada en: ./hyperparameter_tuning_results/Tropical/NR_25046/tuning_table_EVAP.png
    Optimizar Hiperparámetros para: TMAX...


       [OK] Mejor RMSE: 3.0753
       -> Tabla guardada en: ./hyperparameter_tuning_results/Tropical/NR_25046/tuning_table_TMAX.png
    Optimizar Hiperparámetros para: TMIN...


       [OK] Mejor RMSE: 2.4858
       -> Tabla guardada en: ./hyperparameter_tuning_results/Tropical/NR_25046/tuning_table_TMIN.png

--- TUNING COMPLETADO ---


## Performance Metrics

#### PROPHET Performance Metrics

In [6]:
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import logging

#Directorio de tus datos originales, Hiperparametros y carpetas de salida
directory = {
    25037: {
        'Clasificacion': 'Seco', 
        'input_path': "../data/data_conagua_clasificada/Seco/NR_25037.csv",
        'output_folder': "./Performance_metrics_plots/seco/NR_25037/EVAP/Daily",
        'hiperparametros':{
            'EVAP':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 0.01
            },
            'PRECIP':{
                'changepoint_prior_scale': 0.5,
                'seasonality_prior_scale': 0.1
            },
            'TMAX':{
                'changepoint_prior_scale': 0.5,
                'seasonality_prior_scale': 1.0
            },
            'TMIN':{
                'changepoint_prior_scale': 0.5,
                'seasonality_prior_scale': 0.01
        }
        }
    
},    25033: {
        'Clasificacion': 'Templado', 
        'input_path': "../data/data_conagua_clasificada/Templado/NR_25033.csv",
        'output_folder': "./Performance_metrics_plots/templado/NR_25033/EVAP/Daily",
        'hiperparametros':{
            'EVAP':{
                'changepoint_prior_scale': 0.5,
                'seasonality_prior_scale': 0.01
            },
            'PRECIP':{
                'changepoint_prior_scale': 0.05,
                'seasonality_prior_scale': 1.0
            },
            'TMAX':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 0.01
            },
            'TMIN':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 0.01
        }
        },
        
},    25046: {
        'Clasificacion': 'Tropical', 
        'input_path': "../data/data_conagua_clasificada/tropical/NR_25046.csv",
        'output_folder': "./Performance_metrics_plots/tropical/NR_25046/EVAP/Daily",
        'hiperparametros':{
            'EVAP':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 5.0
            },
            'PRECIP':{
                'changepoint_prior_scale': 0.1,
                'seasonality_prior_scale': 10.0
            },
            'TMAX':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 1.0
            },
            'TMIN':{
                'changepoint_prior_scale': 0.001,
                'seasonality_prior_scale': 1.0
        }
        }
    }
}

columnas_objetivo = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

for estacion, info in directory.items():
    file_route = info['input_path']
    output_route = info['output_folder']
    os.makedirs(output_route, exist_ok=True)
    print(f"--> Procesando Estación: {estacion} ({file_route})")
    
    for columna in columnas_objetivo:
        col_output_route = os.path.join(output_route, columna)
        os.makedirs(col_output_route, exist_ok=True)
        print(f"    Procesando variable: {columna}")
        params = info['hiperparametros'][columna]
        print(f"    Usando hiperparámetros: {params}")
        
        all_params = [dict(zip(params.keys(), v)) for v in itertools.product(*[ [v] for v in params.values() ])]
        df = pd.read_csv(file_route)
        
        df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
        df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
        df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
        df_prop = df_prop.dropna(subset=['ds','y']).sort_values('ds').reset_index(drop=True)
        for params in all_params:
            m = Prophet(**params)
            try:
                m = Prophet(**params).fit(df_prop)
                
                
                df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
                
                display(df_cv.head())
                
                
                df_p = performance_metrics(df_cv)
                print(df_p.head())
                
                n_rows = 40
                display_df = df_p.head(n_rows).copy()
                for c in display_df.select_dtypes(include=[float]).columns:
                    display_df[c] = display_df[c].map(lambda x: f"{x:.4f}" if pd.notnull(x) and np.isfinite(x) else "")
                rows, cols = display_df.shape
                
                # Calcular tamaño dinámico de la figura
                fig_w = max(8, min(24, 1.2 * cols))
                fig_h = max(2, 0.3 * rows)
                fig, ax = plt.subplots(figsize=(fig_w, fig_h))
                ax.axis('off')
                table = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
                table.auto_set_font_size(False)
                table.set_fontsize(9)
                table.scale(1, 1.2)
                png_file = os.path.join(col_output_route, f"{estacion}_{columna}_performance_metrics.png")
                fig.tight_layout()
                fig.savefig(png_file, dpi=150, bbox_inches='tight')
                plt.close(fig)
                print(f"  Tabla de métricas de desempeño guardada en: {png_file}")
                
                
            except Exception as e:
                print(f"Fallo con parámetros {params}: {e}")



--> Procesando Estación: 25037 (../data/data_conagua_clasificada/Seco/NR_25037.csv)
    Procesando variable: PRECIP
    Usando hiperparámetros: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,2.751463,-5.235218,10.449269,0.0,1958-01-19
1,1959-01-02,2.055270,-6.167869,10.601404,0.0,1958-01-19
2,1959-01-03,2.099441,-6.022478,9.821681,0.0,1958-01-19
3,1959-01-04,2.530893,-5.679546,11.439272,0.0,1958-01-19
4,1959-01-05,2.870535,-5.060247,10.981790,0.5,1958-01-19


  horizon        mse      rmse       mae  mdape     smape  coverage
0 37 days  53.601518  7.321306  2.675920    inf  1.871068  0.958417
1 38 days  52.497024  7.245483  2.658990    inf  1.871099  0.958438
2 39 days  52.457342  7.242744  2.658133    inf  1.872273  0.958421
3 40 days  51.775913  7.195548  2.639641    inf  1.871578  0.958929
4 41 days  51.742000  7.193191  2.642877    inf  1.871946  0.959002
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/EVAP/Daily\PRECIP\25037_PRECIP_performance_metrics.png
    Procesando variable: EVAP
    Usando hiperparámetros: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01}


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1959-01-01,4.576430,2.240168,6.783402,2.17,1958-06-16
1,1959-01-02,4.389354,1.983031,6.921913,3.89,1958-06-16
2,1959-01-03,4.365116,2.071795,6.699420,4.12,1958-06-16
3,1959-01-04,4.297404,1.795850,6.634858,1.78,1958-06-16
4,1959-01-05,4.034178,1.597441,6.321166,0.61,1958-06-16


  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  3.887949  1.971788  1.469979  0.387861  0.178325  0.249272   
1 38 days  3.884563  1.970930  1.464925  0.385529  0.176878  0.247812   
2 39 days  3.863215  1.965506  1.458024  0.388261  0.175234  0.246988   
3 40 days  3.851121  1.962427  1.453908  0.389067  0.173019  0.246226   
4 41 days  3.865132  1.965994  1.455690  0.393202  0.172577  0.246861   

   coverage  
0  0.829757  
1  0.830268  
2  0.831703  
3  0.832687  
4  0.833754  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/seco/NR_25037/EVAP/Daily\EVAP\25037_EVAP_performance_metrics.png
    Procesando variable: TMAX
    Usando hiperparámetros: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0}


...

  horizon       mse      rmse       mae      mape     mdape     smape  \
0 38 days  2.491711  1.578515  1.140898  0.340168  0.167691  0.244543   
1 39 days  2.469225  1.571377  1.137609  0.339417  0.167749  0.244155   
2 40 days  2.453205  1.566271  1.134809  0.339860  0.167806  0.244115   
3 41 days  2.439826  1.561994  1.132221  0.337827  0.167691  0.244227   
4 42 days  2.429972  1.558837  1.129093  0.337308  0.167080  0.243513   

   coverage  
0  0.861871  
1  0.863110  
2  0.864380  
3  0.864281  
4  0.864066  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/tropical/NR_25046/EVAP/Daily\EVAP\25046_EVAP_performance_metrics.png
    Procesando variable: TMAX
    Usando hiperparámetros: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0}


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1944-05-04,36.381087,33.200883,39.957308,35.0,1944-05-03
1,1944-05-05,36.933814,33.375757,40.447325,38.0,1944-05-03
2,1944-05-06,36.818390,33.213908,40.026231,37.0,1944-05-03
3,1944-05-07,37.182995,33.575594,40.631380,38.0,1944-05-03
4,1944-05-08,37.433251,33.980350,41.060838,40.0,1944-05-03


  horizon       mse      rmse       mae      mape     mdape     smape  \
0 38 days  9.797831  3.130149  2.355264  0.076495  0.054202  0.073917   
1 39 days  9.733041  3.119782  2.348611  0.076192  0.054039  0.073669   
2 40 days  9.704238  3.115163  2.345315  0.076073  0.053740  0.073561   
3 41 days  9.684277  3.111957  2.344429  0.076031  0.053740  0.073532   
4 42 days  9.742328  3.121270  2.352155  0.076358  0.053807  0.073819   

   coverage  
0  0.823786  
1  0.824815  
2  0.825078  
3  0.825239  
4  0.823748  
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/tropical/NR_25046/EVAP/Daily\TMAX\25046_TMAX_performance_metrics.png
    Procesando variable: TMIN
    Usando hiperparámetros: {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0}


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,1944-05-04,15.138538,11.804497,18.562705,15.0,1944-05-03
1,1944-05-05,15.490475,11.901862,19.226601,15.5,1944-05-03
2,1944-05-06,15.366153,11.698824,18.992963,16.0,1944-05-03
3,1944-05-07,15.388294,11.580527,19.059327,15.0,1944-05-03
4,1944-05-08,15.153351,11.725791,18.902223,15.0,1944-05-03


  horizon       mse      rmse       mae     mdape     smape  coverage
0 38 days  6.268624  2.503722  1.922242  0.095477  0.160750  0.824197
1 39 days  6.286285  2.507246  1.923941  0.095018  0.160899  0.823246
2 40 days  6.255362  2.501072  1.920277  0.095531  0.160438  0.825207
3 41 days  6.230042  2.496005  1.915404  0.095399  0.160014  0.825902
4 42 days  6.239470  2.497893  1.915557  0.095257  0.160087  0.825925
  Tabla de métricas de desempeño guardada en: ./Performance_metrics_plots/tropical/NR_25046/EVAP/Daily\TMIN\25046_TMIN_performance_metrics.png


#### Custom Performance Metrics (Horizontes)

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation
import matplotlib.pyplot as plt
import joblib
import os
import warnings
import logging

# Silenciar logs ruidosos
warnings.filterwarnings('ignore')
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)

# --- CONFIGURACIÓN DE RUTAS Y PARÁMETROS ---

# Estructura centralizada de rutas
directory = {
    '25037': {
        'Clasificacion': 'Seco',
        'input_csv': "../data/data_conagua_clasificada/Seco/NR_25037.csv",
        # Ruta base donde están tus modelos .joblib guardados
        'models_base_dir': "./saved_models/FBProphet/Seco", 
        'hiperparametros': {
            'EVAP': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01},
            'PRECIP': {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1},
            'TMAX': {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0},
            'TMIN': {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.01}
        }
    },
    '25033': {
        'Clasificacion': 'Templado',
        'input_csv': "../data/data_conagua_clasificada/Templado/NR_25033.csv",
        'models_base_dir': "./saved_models/FBProphet/Templado",
        'hiperparametros': {
            'EVAP': {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.01},
            'PRECIP': {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 1.0},
            'TMAX': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01},
            'TMIN': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 0.01}
        }
    },
    '25046': {
        'Clasificacion': 'Tropical',
        'input_csv': "../data/data_conagua_clasificada/Tropical/NR_25046.csv",
        'models_base_dir': "./saved_models/FBProphet/Tropical",
        'hiperparametros': {
            'EVAP': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 5.0},
            'PRECIP': {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 10.0},
            'TMAX': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0},
            'TMIN': {'changepoint_prior_scale': 0.001, 'seasonality_prior_scale': 1.0}
        }
    }
}

columnas = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

# Bins de tiempo para el análisis
bins = [0, 3, 7, 31, 180, 365, 730, float('inf')]
labels = ['1-3 Dias', '4-7 Dias', '8-31 Dias', '32-180 Dias', '181-365 Dias', '1-2 Años', '>2 Años']

print("--- INICIANDO VALIDACIÓN PROPHET (RUTAS CORREGIDAS) ---")

for estacion_id, info in directory.items():
    region = info['Clasificacion']
    file_csv = info['input_csv']
    models_dir = info['models_base_dir']

    output_base_station = f"./data_analysis/Performance_metrics_plots/FBProphet_VAL/{region}/NR_{estacion_id}/"
    
    print(f"\n--> Procesando Estación: {estacion_id} ({region})")
    
    if not os.path.exists(file_csv):
        print(f"    [SKIP] No existe el archivo de datos: {file_csv}")
        continue

    try:
        df = pd.read_csv(file_csv)
    except Exception as e:
        print(f"    [ERROR] No se pudo leer el CSV: {e}")
        continue

    for columna in columnas:
        print(f"    Evaluando variable: {columna}")
        
        # 1. CREAR CARPETA DE SALIDA ESPECÍFICA PARA LA VARIABLE
        path_salida_final = os.path.join(output_base_station, columna)
        os.makedirs(path_salida_final, exist_ok=True)
        
        # 2. PREPARAR DATOS
        df_prop = df[['FECHA', columna]].rename(columns={'FECHA':'ds', columna:'y'}).copy()
        df_prop['ds'] = pd.to_datetime(df_prop['ds'], errors='coerce')
        df_prop['y'] = pd.to_numeric(df_prop['y'], errors='coerce')
        df_prop = df_prop.dropna().sort_values('ds').reset_index(drop=True)
        
        if len(df_prop) < 100:
            print("      [SKIP] Pocos datos.")
            continue

        # 3. CARGAR O ENTRENAR MODELO
        params = info['hiperparametros'].get(columna)
        if not params: continue
        
        # Nombre del archivo del modelo
        fname_model = f"prophet_cp{params['changepoint_prior_scale']}_sp{params['seasonality_prior_scale']}.joblib"

        path_model_var = os.path.join(models_dir, columna, fname_model)
        path_model_root = os.path.join(models_dir, fname_model)
        
        m = None
        if os.path.exists(path_model_var):
            m = joblib.load(path_model_var)
        elif os.path.exists(path_model_root):
            m = joblib.load(path_model_root)
        else:
            print(f"      [AVISO] Modelo no encontrado en disco. Entrenando temporalmente...")
            m = Prophet(**params)
            m.fit(df_prop)

        # 4. CROSS VALIDATION
        try:
            df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days', parallel="processes")
            
            # Calcular horizontes
            df_cv['horizon_days'] = (df_cv['ds'] - df_cv['cutoff']).dt.total_seconds() / (24 * 3600)
            df_cv['range'] = pd.cut(df_cv['horizon_days'], bins=bins, labels=labels)
            
            # Función de métricas
            def get_metrics(group):
                y_true = group['y']
                y_pred = group['yhat']
                y_safe = y_true.replace(0, np.nan)
                mse = np.mean((y_true - y_pred)**2)
                rmse = np.sqrt(mse)
                mae = np.mean(np.abs(y_true - y_pred))
                mape = np.mean(np.abs((y_true - y_pred) / y_safe))
                return pd.Series({'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'Count': len(y_true)})

            df_metrics = df_cv.groupby('range', observed=False).apply(get_metrics).reset_index()

            
            # A. CSV Métricas
            csv_file = os.path.join(path_salida_final, "metrics_validation.csv")
            df_metrics.to_csv(csv_file, index=False)
            
            # B. Tabla Visual
            display_df = df_metrics.copy()
            for c in ['MSE', 'RMSE', 'MAE', 'MAPE']:
                display_df[c] = display_df[c].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "-")
            
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.axis('off')
            tbl = ax.table(cellText=display_df.values, colLabels=display_df.columns, loc='center', cellLoc='center')
            tbl.scale(1, 1.4)
            plt.title(f'Validación Prophet: {columna} - {estacion_id}')
            plt.savefig(os.path.join(path_salida_final, "tabla_metricas.png"), bbox_inches='tight')
            plt.close()
            
            # C. Gráfica Barras RMSE
            plt.figure(figsize=(10, 6))
            plot_data = df_metrics[df_metrics['Count'] > 0]
            if not plot_data.empty:
                bars = plt.bar(plot_data['range'], plot_data['RMSE'], color='skyblue', edgecolor='black')
                plt.title(f'RMSE por Horizonte - {columna} (Prophet)')
                plt.xticks(rotation=45)
                for bar in bars:
                    height = bar.get_height()
                    plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom')
            plt.savefig(os.path.join(path_salida_final, "RMSE_bar_plot.png"), bbox_inches='tight')
            plt.close()
            
            # D. Gráfica Línea Comparativa
            df_cv_sorted = df_cv.sort_values('ds')
            subset = df_cv_sorted.tail(150)
            plt.figure(figsize=(12, 5))
            plt.plot(subset['ds'], subset['y'], label='Real', color='blue', alpha=0.6)
            plt.plot(subset['ds'], subset['yhat'], label='Predicción Prophet', color='orange', linestyle='--', alpha=0.8)
            plt.title(f'Real vs Pronóstico (CV) - {columna} - {estacion_id}')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(path_salida_final, "plot_linea_comparativa.png"), bbox_inches='tight')
            plt.close()
            
            print(f"      [OK] Guardado en: {path_salida_final}")

        except Exception as e:
            print(f"      [ERROR CRÍTICO] en {columna}: {e}")

print("\n--- PROCESO TERMINADO ---")

--- INICIANDO VALIDACIÓN PROPHET (RUTAS CORREGIDAS) ---

--> Procesando Estación: 25037 (Seco)
    Evaluando variable: PRECIP
      [OK] Guardado en: ./data_analysis/Performance_metrics_plots/FBProphet_VAL/Seco/NR_25037/PRECIP
    Evaluando variable: EVAP
      [OK] Guardado en: ./data_analysis/Performance_metrics_plots/FBProphet_VAL/Seco/NR_25037/EVAP
    Evaluando variable: TMAX
      [OK] Guardado en: ./data_analysis/Performance_metrics_plots/FBProphet_VAL/Seco/NR_25037/TMAX
    Evaluando variable: TMIN
      [OK] Guardado en: ./data_analysis/Performance_metrics_plots/FBProphet_VAL/Seco/NR_25037/TMIN

--> Procesando Estación: 25033 (Templado)
    Evaluando variable: PRECIP
      [AVISO] Modelo no encontrado en disco. Entrenando temporalmente...
      [OK] Guardado en: ./data_analysis/Performance_metrics_plots/FBProphet_VAL/Templado/NR_25033/PRECIP
    Evaluando variable: EVAP
      [AVISO] Modelo no encontrado en disco. Entrenando temporalmente...
      [OK] Guardado en: ./data_anal